<a href="https://colab.research.google.com/github/ysg1202/automoblile/blob/main/0709_OpenCV_MOG2_%EC%B0%A8%EB%9F%89%26%EC%B0%A8%EC%84%A0%EA%B0%90%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

배경 차분기

In [ ]:
# 1. 기본 MOG2 차량 감지 코드
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

# 동영상 파일 열기 (코랩에서는 업로드한 파일 경로 사용)
cap = cv2.VideoCapture('/content/driving.mp4')  # 파일 경로 수정 필요

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"📌 전체 프레임 수: {total_frames}")

# MOG2 배경 차분기 생성
backSub = cv2.createBackgroundSubtractorMOG2()

frame_count = 0
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 배경 차분 적용
        fgMask = backSub.apply(frame) # 흰색 부분은 움직임이 있는 부분, 검정색 부분은 배경

        # 매 30프레임마다 결과 출력 (너무 많은 출력 방지)
        if frame_count % 30 == 0:   # 30초마다 1프레임
            # 결과를 나란히 표시
            combined = np.hstack((frame, cv2.cvtColor(fgMask, cv2.COLOR_GRAY2BGR)))
            cv2_imshow(combined)

        frame_count += 1

        # 100프레임 정도만 처리 (테스트용)
        if frame_count > 100:
            break


# 자원 해제
# cap.release() # 안하면 파일이 "사용중" 상태
# print("처리 완료!")

finally:
    # 항상 실행됨
    cap.release()
    # cv2.destroyAllWindows()
    print("자원 해제 완료!")

노이즈 제거 기능 추가

In [ ]:
# 1. 기본 MOG2 차량 감지 코드
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt


# 동영상 파일 열기 (코랩에서는 업로드한 파일 경로 사용)
cap = cv2.VideoCapture('/content/driving.mp4')  # 파일 경로 수정 필요


# MOG2 배경 차분기 생성
backSub = cv2.createBackgroundSubtractorMOG2()


frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 배경 차분 적용
    fgMask = backSub.apply(frame)

    # 노이즈 제거 (모폴로지 연산)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)) # 연산 모양과 크기 결정 (여기선 타원형 3x3)
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_OPEN, kernel)  # 작은 흰 점(노이즈) 제거
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_CLOSE, kernel)  # 구멍 메우기

    # 매 30프레임마다 결과 출력 (너무 많은 출력 방지)
    if frame_count % 10 == 0:   # 10초마다 1프레임
        # 결과를 나란히 표시
        combined = np.hstack((frame, cv2.cvtColor(fgMask, cv2.COLOR_GRAY2BGR)))
        cv2_imshow(combined)

    frame_count += 1

    # 100프레임 정도만 처리 (테스트용)
    if frame_count > 100:
        break


# 자원 해제
cap.release()
print("처리 완료!")

바운딩 박스 추가

In [ ]:
# 1. 기본 MOG2 차량 감지 코드
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt


# 동영상 파일 열기 (코랩에서는 업로드한 파일 경로 사용)
cap = cv2.VideoCapture('/content/driving.mp4')  # 파일 경로 수정 필요


# MOG2 배경 차분기 생성
backSub = cv2.createBackgroundSubtractorMOG2()


frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 배경 차분 적용
    fgMask = backSub.apply(frame)

    # 노이즈 제거 (모폴로지 연산)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)) # 연산 모양과 크기 결정 (여기선 타원형 3x3)
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_OPEN, kernel)  # 작은 흰 점(노이즈) 제거
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_CLOSE, kernel)  # 구멍 메우기

    # 윤곽선 검출 및 바운딩 박스
    contours, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 바운딩 박스를 그릴 프레임 복사
    result_frame = frame.copy()

    for contour in contours:
        # 너무 작은 영역은 제외 (차량이 아닐 가능성 높음)
        if cv2.contourArea(contour) > 3000:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(result_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)


    # 매 30프레임마다 결과 출력 (너무 많은 출력 방지)
    if frame_count % 10 == 0:   # 10초마다 1프레임
        # 결과를 나란히 표시
        combined = np.hstack((frame, cv2.cvtColor(fgMask, cv2.COLOR_GRAY2BGR)))
        cv2_imshow(combined)

    frame_count += 1

    # 100프레임 정도만 처리 (테스트용)
    if frame_count > 100:
        break


# 자원 해제
cap.release()
print("처리 완료!")

종횡비 추가 (아직안함)

In [ ]:
# 1. 기본 MOG2 차량 감지 코드
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt


# 동영상 파일 열기 (코랩에서는 업로드한 파일 경로 사용)
cap = cv2.VideoCapture('/content/driving.mp4')  # 파일 경로 수정 필요


# MOG2 배경 차분기 생성
backSub = cv2.createBackgroundSubtractorMOG2()


frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 배경 차분 적용
    fgMask = backSub.apply(frame)

    # 노이즈 제거 (모폴로지 연산)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)) # 연산 모양과 크기 결정 (여기선 타원형 3x3)
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_OPEN, kernel)  # 작은 흰 점(노이즈) 제거
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_CLOSE, kernel)  # 구멍 메우기

    # 윤곽선 검출 및 바운딩 박스
    contours, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 바운딩 박스를 그릴 프레임 복사
    result_frame = frame.copy()

    for contour in contours:
        # 너무 작은 영역은 제외 (차량이 아닐 가능성 높음)
        if cv2.contourArea(contour) > 4000:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(result_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)


    # 매 30프레임마다 결과 출력 (너무 많은 출력 방지)
    if frame_count % 10 == 0:   # 10초마다 1프레임
        # 결과를 나란히 표시
        #combined = np.hstack((frame, cv2.cvtColor(fgMask, cv2.COLOR_GRAY2BGR))) # 수정
        combined = np.hstack((result_frame, cv2.cvtColor(fgMask, cv2.COLOR_GRAY2BGR)))
        cv2_imshow(combined)

    frame_count += 1

    # 100프레임 정도만 처리 (테스트용)
    if frame_count > 100:
        break


# 자원 해제
cap.release()
print("처리 완료!")

ROI 추가

In [ ]:
# 1. 기본 MOG2 차량 감지 코드
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt


# 동영상 파일 열기 (코랩에서는 업로드한 파일 경로 사용)
cap = cv2.VideoCapture('/content/driving.mp4')  # 파일 경로 수정 필요


# MOG2 배경 차분기 생성
backSub = cv2.createBackgroundSubtractorMOG2()

# ROI 설정 (도로 영역만 분석) - 좌표는 영상에 맞게 조정 필요
def create_roi_mask(frame):
    height, width = frame.shape[:2] # (높이, 너비, 채널) → 앞의 두 값만 사용
    mask = np.zeros((height, width), dtype=np.uint8)

    # 도로 영역을 다각형으로 설정 (예시 - 실제 영상에 맞게 조정)
    roi_points = np.array([
        [0, height//2],           # 왼쪽 중간
        [width, height//2],       # 오른쪽 중간
        [width, height],          # 오른쪽 아래
        [0, height]               # 왼쪽 아래
    ], np.int32)

    cv2.fillPoly(mask, [roi_points], 255) # 지정된 다각형 영역을 **흰색(255)**으로 채움
    return mask


frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 배경 차분 적용
    fgMask = backSub.apply(frame)

     # ROI 마스크 생성 및 적용
    roi_mask = create_roi_mask(frame)
    fgMask = cv2.bitwise_and(fgMask, roi_mask)  # ROI 영역만 남김


    # 노이즈 제거 (모폴로지 연산)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)) # 연산 모양과 크기 결정 (여기선 타원형 3x3)
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_OPEN, kernel)  # 작은 흰 점(노이즈) 제거
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_CLOSE, kernel)  # 구멍 메우기

    # 윤곽선 검출 및 바운딩 박스
    contours, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 바운딩 박스를 그릴 프레임 복사
    result_frame = frame.copy()

    for contour in contours:
        # 너무 작은 영역은 제외 (차량이 아닐 가능성 높음)
        if cv2.contourArea(contour) > 4000:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(result_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)


    # 매 30프레임마다 결과 출력 (너무 많은 출력 방지)
    if frame_count % 10 == 0:   # 10초마다 1프레임
        # 결과를 나란히 표시
        #combined = np.hstack((frame, cv2.cvtColor(fgMask, cv2.COLOR_GRAY2BGR))) # 수정1
        combined = np.hstack((result_frame, cv2.cvtColor(fgMask, cv2.COLOR_GRAY2BGR)))
        cv2_imshow(combined)

    frame_count += 1

    # 100프레임 정도만 처리 (테스트용)
    if frame_count > 100:
        break


# 자원 해제
cap.release()
print("처리 완료!")

사다리꼴 ROI로 변경, 최종 종합

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# 동영상 파일 열기
cap = cv2.VideoCapture('/content/driving.mp4')  # 파일 경로 수정 필요

# 차선 검출을 위한 ROI 설정
def create_lane_roi(frame):
    height, width = frame.shape[:2]
    mask = np.zeros((height, width), dtype=np.uint8)

    #offset = 60

    roi_points = np.array([
    [width//4 - 80, height],        # 왼쪽 아래 (중앙 기준 좌측)
    [width//2 - 10, height//2 +60],  # 왼쪽 위
    [width//2 + 170, height//2 + 60],  # 오른쪽 위
    [width*3//4 + 250, height]         # 오른쪽 아래 (중앙 기준 우측)
], np.int32)



    cv2.fillPoly(mask, [roi_points], 255)
    return mask, roi_points

frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 그레이스케일 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 가우시안 블러로 노이즈 제거
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Canny 에지 검출
    edges = cv2.Canny(blur, 50, 150)

    # ROI 적용
    roi_mask, roi_points = create_lane_roi(frame)
    masked_edges = cv2.bitwise_and(edges, roi_mask)

    # 결과 프레임 복사
    result_frame = frame.copy()

    cv2.polylines(result_frame, [roi_points], isClosed=True, color=(0, 255, 0), thickness=2)

    # 매 30프레임마다 결과 출력
    if frame_count % 10 == 0:
        # 결과를 나란히 표시 (원본 + 에지)
        combined = np.hstack((result_frame, cv2.cvtColor(masked_edges, cv2.COLOR_GRAY2BGR)))
        cv2_imshow(combined)

    frame_count += 1

    # 100프레임 후 종료 (테스트용)
    if frame_count > 100:
        break

# 자원 해제
cap.release()

차선 검출

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# 동영상 파일 열기
cap = cv2.VideoCapture('/content/driving.mp4')  # 파일 경로 수정 필요

# 차선 검출을 위한 ROI 설정
def create_lane_roi(frame):
    height, width = frame.shape[:2]
    mask = np.zeros((height, width), dtype=np.uint8)

    # 사다리꼴 모양의 ROI (차선이 있는 도로 영역)
    roi_points = np.array([
    [width//4 - 80, height],        # 왼쪽 아래 (중앙 기준 좌측)
    [width//2 - 10, height//2 +60],  # 왼쪽 위
    [width//2 + 170, height//2 + 60],  # 오른쪽 위
    [width*3//4 + 250, height]         # 오른쪽 아래 (중앙 기준 우측)
], np.int32)


    cv2.fillPoly(mask, [roi_points], 255)
    return mask

frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 그레이스케일 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 가우시안 블러로 노이즈 제거
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Canny 에지 검출
    edges = cv2.Canny(blur, 20, 120)

    # ROI 적용
    roi_mask = create_lane_roi(frame)
    masked_edges = cv2.bitwise_and(edges, roi_mask)

    # Hough 변환으로 직선 검출
    lines = cv2.HoughLinesP(masked_edges, 1, np.pi/180, threshold=30,
                           minLineLength=50, maxLineGap=70)

    # 결과 프레임들 복사
    result_frame = frame.copy()
    lane_frame = frame.copy()

    # 차선 분류를 위한 리스트
    left_lines = []
    right_lines = []

    # 검출된 차선을 좌/우로 분류
    if lines is not None:
        img_center = frame.shape[1] // 2  # 화면 중앙

        for line in lines:
            x1, y1, x2, y2 = line[0]

            # 기울기 계산 (수직선 제외)
            if x2 - x1 != 0:
                slope = (y2 - y1) / (x2 - x1)

                # 기울기와 위치로 좌/우 차선 분류
                if slope < -0.3 and x1 < img_center:  # 왼쪽 차선 (음의 기울기)
                    left_lines.append([x1, y1, x2, y2])
                elif slope > 0.3 and x1 > img_center:  # 오른쪽 차선 (양의 기울기)
                    right_lines.append([x1, y1, x2, y2])

    # 차선 연결 함수
    def connect_lane_segments(lines, frame_height):
        if not lines:
            return None

        # 모든 점들을 수집
        points = []
        for x1, y1, x2, y2 in lines:
            points.extend([(x1, y1), (x2, y2)])

        if len(points) < 2:
            return None

        # 최소자승법으로 직선 피팅
        points = np.array(points)
        x_coords = points[:, 0]
        y_coords = points[:, 1]

        # 1차 다항식 피팅
        coeffs = np.polyfit(y_coords, x_coords, 1)

        # 화면 상하단에서의 x 좌표 계산
        y_top = frame_height // 2
        y_bottom = frame_height
        x_top = int(coeffs[0] * y_top + coeffs[1])
        x_bottom = int(coeffs[0] * y_bottom + coeffs[1])

        return [x_top, y_top, x_bottom, y_bottom]

    # 좌측/우측 차선 연결
    left_lane = connect_lane_segments(left_lines, frame.shape[0])
    right_lane = connect_lane_segments(right_lines, frame.shape[0])

    # 연결된 차선 그리기
    if left_lane:
        cv2.line(lane_frame, (left_lane[0], left_lane[1]),
                (left_lane[2], left_lane[3]), (0, 255, 0), 5)  # 녹색 왼쪽 차선
        cv2.putText(lane_frame, "LEFT", (left_lane[0]-30, left_lane[1]-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    if right_lane:
        cv2.line(lane_frame, (right_lane[0], right_lane[1]),
                (right_lane[2], right_lane[3]), (255, 0, 0), 5)  # 파란색 오른쪽 차선
        cv2.putText(lane_frame, "RIGHT", (right_lane[0]+10, right_lane[1]-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    # ROI 영역을 시각화 (사다리꼴 모양 표시)
    roi_points = np.array([
        [frame.shape[1]//4, frame.shape[0]],        # 왼쪽 아래
        [frame.shape[1]*7//10, frame.shape[0]*4//5], # 왼쪽 위
        [frame.shape[1]*6//8, frame.shape[0]*3//5], # 오른쪽 위
        [frame.shape[1]*9//10, frame.shape[0]]      # 오른쪽 아래
    ], np.int32)

    cv2.polylines(result_frame, [roi_points], True, (255, 0, 0), 2)  # 파란색 사다리꼴

    # 매 30프레임마다 결과 출력
    if frame_count % 10 == 0:
        # 영어 라벨 추가
        cv2.putText(result_frame, "Original + ROI", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        edge_display = cv2.cvtColor(masked_edges, cv2.COLOR_GRAY2BGR)
        cv2.putText(edge_display, "Canny Edge Detection", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        cv2.putText(lane_frame, "Lane Detection Result", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # 3개 화면을 나란히 표시
        combined = np.hstack((result_frame, edge_display, lane_frame))
        cv2_imshow(combined)

    frame_count += 1

    # 100프레임 후 종료 (테스트용)
    if frame_count > 100:
        break

# 자원 해제
cap.release()
print("차선 검출 처리 완료!")